In [75]:
import pandas as pd
import numpy as np
import os

In [76]:
train_data = pd.read_csv("../content/train_cleaned.csv")
test_data = pd.read_csv("../content/test_cleaned.csv")

In [77]:
train_data.head()

,Unnamed: 0,textID,text,selected_text,sentiment
0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
1,2,088c60f138,my boss is bullying me...,bullying me,negative
2,3,9642c003ef,what interview! leave me alone,leave me alone,negative
3,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
4,6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive


In [78]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16363 entries, 0 to 16362
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     16363 non-null  int64 
 1   textID         16363 non-null  object
 2   text           16363 non-null  object
 3   selected_text  16363 non-null  object
 4   sentiment      16363 non-null  object
dtypes: int64(1), object(4)
memory usage: 639.3+ KB


In [79]:
X_train= train_data.loc[:10000, 'text']
X_test = test_data.loc[:10000, 'text']

In [80]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train = le.fit_transform(train_data.loc[:10000, 'sentiment'])
Y_test = le.transform(test_data.loc[:10000, 'sentiment'])

In [81]:
Y_train

array([0, 0, 0, ..., 0, 0, 0])

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(lowercase = True, stop_words = 'english') #min_df
X_train1 = tf.fit_transform(X_train)
X_test1 = tf.transform(X_test)

In [83]:
X_train1 = pd.DataFrame(X_train1.toarray())
X_test1 = pd.DataFrame(X_test1.toarray())

In [84]:
print("X_train1:")
print(X_train1)

X_train1:
       0      1      2      3      4      5      6      7      8      9      \
0        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
9996     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9997     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9998     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9999     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
10000    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

       ...  13599  13600  13601  13602  1

In [85]:
def sigmoid(z):
  return 1 / (1+np.exp(-z))
  #you should initialise with very small values in case of sigmoid function due to vanishing gradient problem

In [86]:
def training_phase(X, Y, lr, iterations):
  n = len(X)
  X = X.T
  Y = Y.T
  W = np.random.randint(-10, 10, size = (X.shape[0], 1))*0.01 #0.01 is done in case of sigmoid or tanh due to vanishing gradient
  b = 0
  for i in range(iterations):
    #Forward Propagation
    A = sigmoid(np.dot(W.T, X)+ b)
    cost = (-1/n) * np.sum(Y * np.log(A) + (1-Y) * np.log(1-A)) #Hyperparameter tuning is done on basis of cost (Binary Cross Entropy)
    print('iterations:' + str(i)+ ',' + "cost:" + str(cost))
    #Backward Propagation
    dW = (1/n)*np.dot(X, (A-Y).T)
    db = (1/n)*np.sum((A-Y).T)
    W = W - lr*dW
    b = b - lr*db
  return W, b




In [95]:
W, b = training_phase(X_train1, Y_train, 10, 1000)

iterations:0,cost:0.6949362398613481
iterations:1,cost:0.6927777606902309
iterations:2,cost:0.6935799500168746
iterations:3,cost:0.7010258241156482
iterations:4,cost:0.7201614501492875
iterations:5,cost:0.7638153165723007
iterations:6,cost:0.8255516697230285
iterations:7,cost:0.9201993633161787
iterations:8,cost:0.9484765613057192
iterations:9,cost:1.015800408014184
iterations:10,cost:0.9653315085998844
iterations:11,cost:1.018901544642556
iterations:12,cost:0.9573783629839018
iterations:13,cost:1.0092768522557094
iterations:14,cost:0.9483582975083122
iterations:15,cost:0.9990008752822033
iterations:16,cost:0.9393379815077214
iterations:17,cost:0.9885926594100921
iterations:18,cost:0.9303589477252189
iterations:19,cost:0.9780952292580786
iterations:20,cost:0.9214249819597639
iterations:21,cost:0.9675337519750443
iterations:22,cost:0.9125373267497386
iterations:23,cost:0.956931362874965
iterations:24,cost:0.9036963458650237
iterations:25,cost:0.946309479805473
iterations:26,cost:0.89490

In [96]:
def testing_phase(W, b, X):
  Y_predict =  np.zeros((1, X.shape[0]))
  A = sigmoid(np.dot(W.T, X.T) + b)
  for i in range(A.shape[1]):
    if (A[0,i] > 0.5):
      Y_predict[0,i] = 1
  return Y_predict



In [97]:
Y_predict = testing_phase(W, b, X_test1)
Y_predict

array([[1., 0., 1., ..., 0., 1., 1.]])

In [98]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_predict.flatten())

0.8716730038022814